In [ ]:
import math
from pomegranate import *

In [ ]:
import pygraphviz
import tempfile
import matplotlib.pyplot as plt
import matplotlib.image as matimg

def plot(model):
    G = pygraphviz.AGraph(directed=True)
    
    for state in model.states:
        G.add_node(state.name, color='red')
        
    for parent, child in model.edges:
        G.add_edge(parent.name, child.name)
        
    with tempfile.NamedTemporaryFile() as tf:
        G.draw(tf, format='png', prog='dot')
        img = matimg.imread(tf)
        plt.imshow(img)
        plt.axis('off')
        plt.show()

In [ ]:
# Wahrscheinlichkeitstabellen
wk_einbruch = DiscreteDistribution({'e': 0.001, 'ne': 0.999})
wk_erdbeben = DiscreteDistribution({'b': 0.002, 'nb': 0.998})
wk_alarm = ConditionalProbabilityTable([
    ['e', 'b', 'a', 0.95],
    ['e', 'b', 'na', 0.05],
    ['e', 'nb', 'a', 0.94],
    ['e', 'nb', 'na', 0.06],
    ['ne', 'b', 'a', 0.29],
    ['ne', 'b', 'na', 0.71],
    ['ne', 'nb', 'a', 0.001],
    ['ne', 'nb', 'na', 0.999]], [wk_einbruch, wk_erdbeben])
wk_john = ConditionalProbabilityTable([
    ['a', 'j', 0.9],
    ['a', 'nj', 0.1],
    ['na', 'j', 0.05],
    ['na', 'nj', 0.95]], [wk_alarm])
wk_mary = ConditionalProbabilityTable([
    ['a', 'm', 0.7],
    ['a', 'nm', 0.3],
    ['na', 'm', 0.01],
    ['na', 'nm', 0.99]], [wk_alarm])

In [ ]:
einbruch = Node(wk_einbruch, name="Einbruch")
erdbeben = Node(wk_erdbeben, name="Erdbeben")
alarm = Node(wk_alarm, name="Alarm")
john = Node(wk_john, name="John")
mary = Node(wk_mary, name="Mary")



In [ ]:
network = BayesianNetwork("Alarm-BN von Judea Pearl")
network.add_states(einbruch, erdbeben, alarm, john, mary)
network.add_edge(einbruch, alarm)
network.add_edge(erdbeben, alarm)
network.add_edge(alarm, john)
network.add_edge(alarm, mary)
network.bake()

In [ ]:
plot(network)

In [ ]:
p = network.probability([['ne','nb','a','j','m']])
p

In [ ]:
beliefs = network.predict_proba({ 'John' : 'j', 'Mary' : 'm'})
beliefs

In [ ]:
m = network.marginal()
m